In [1]:
!pip install scikit-multiflow --q

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import time

In [2]:
time.time()

1607422801.6473866

In [30]:
'''class dtel:
    def __init__(self, pool_size=4):
        self.pool_size=pool_size
        self.pool = []
        self.new_pool = []
        self.model_pool_weight = []
    
    def train_new_model(self, df, y):
         # update all models
        self.new_pool = copy.deepcopy(self.pool[:])
        
        self.temp_pool = []
        # for all models
        #print('updating models')
        
        for m in self.new_pool:
            start = time.time()
            model = copy.deepcopy(m)
            #freeze all layers except last one
            for i in model.layers[:-1]:
                i.trainable = False
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            model.fit(df, y, epochs=20, batch_size=100, verbose=0)
            print('retrain lasted ', time.time()-start)
            self.temp_pool.append(model)
            keras.backend.clear_session()
            
        # train new model
        #print('training models')
        start = time.time()
        new_model = create_model()
        new_model.fit(df, y, epochs=20, batch_size=100, verbose=0)
        print('train from scratch lasted ', time.time()-start)
        self.temp_pool.append(new_model)
        keras.backend.clear_session()
        # compute weights
        self.model_pool_weight = mse_i_t(self.temp_pool, df)
        
        
    def update_pool(self, df, y):
        if len(self.temp_pool) > self.pool_size:
            self.pool = compute_difference(self.pool, df, y)
        else :
            self.new_pool.extend([self.temp_pool[-1]])
            self.pool = list(self.new_pool[:])

    def predict(self, df):
        y_prob = self.temp_pool[0].predict(df)
        pred = np.array([np.argmax(y_prob[i]) for i in range(len(y_prob))])*self.model_pool_weight[0]
        
        for i in range(1, len(self.temp_pool)):
            y_prob = self.temp_pool[0].predict(df)
            pred = pred+np.array([np.argmax(y_prob[j]) for j in range(len(y_prob))])*self.model_pool_weight[i]

        pred = pred/sum(self.model_pool_weight)
        return pred'''

"class dtel:\n    def __init__(self, pool_size=4):\n        self.pool_size=pool_size\n        self.pool = []\n        self.new_pool = []\n        self.model_pool_weight = []\n    \n    def train_new_model(self, df, y):\n         # update all models\n        self.new_pool = copy.deepcopy(self.pool[:])\n        \n        self.temp_pool = []\n        # for all models\n        #print('updating models')\n        \n        for m in self.new_pool:\n            start = time.time()\n            model = copy.deepcopy(m)\n            #freeze all layers except last one\n            for i in model.layers[:-1]:\n                i.trainable = False\n            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\n            model.fit(df, y, epochs=20, batch_size=100, verbose=0)\n            print('retrain lasted ', time.time()-start)\n            self.temp_pool.append(model)\n            keras.backend.clear_session()\n            \n        # train new model\n      

In [1]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
import time as t
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
import numpy as np
import pandas as pd
import random
from skmultiflow.data import SEAGenerator as sea
import json
import copy

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
def create_model():
    keras.backend.clear_session()
    num_classes = 2
    num_h = 5
    model = Sequential()
    l1 = Flatten(input_shape=(1, 3))
    model.add(l1)
    l2 = Dense(num_h, activation='relu', kernel_initializer='normal')
    model.add(l2)
    model.add(Dense(num_classes, activation='softmax', kernel_initializer='normal'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [37]:
!mkdir models

In [3]:
def mse_i_t(model_pool, df):
    start = time.time()
    w = []
    for i in range(len(model_pool)-1):
        m = keras.models.load_model(model_pool[i])
        w.append(compute_weights(m, df))
    # last model is of weight highest
    w.append(1)
    print('call to mse_i_t lasted', time.time()-start)
    return w

def compute_weights(model, df):
    start = time.time()
    eps = 1
    try:
        df = df.to_numpy()
    except:
        pass
    y = list(row[-1] for row in df)
    y_prob = model.predict(df)
    y_pred = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
    print('call to compute_weights lasted', time.time()-start)
    return 1/(sum((pd.DataFrame(y_pred)[0]-pd.DataFrame(y)[1])**2)/len(df)+eps)



def compute_divS(model_pool_minus_one, df, y):
    start = time.time()
    sum_q = 0
    for i in range(len(model_pool_minus_one)):
        for j in range(i-1):
            m1 = model_pool_minus_one[i]
            m2 = model_pool_minus_one[j]
            sum_q+=Q(df, y, m1, m2)
    print('call to compute_divS lasted', time.time()-start)
    return 1-sum_q/len(model_pool_minus_one)

'''def compute_difference(model_pool, df, y):
    start = time.time()
    vals_q = {}
    for i in model_pool:
        model_pool_temp = model_pool.copy()
        model_pool_temp.remove(i)
        vals_q[i] = compute_divS(model_pool_temp, df, y)
    to_remove = min(vals_q, key=vals_q.get)
    #print('removing ', to_remove)
    model_pool.remove(to_remove)
    print('call to compute_difference lasted', time.time()-start)
    return model_pool'''

def compute_difference(model_pool_path, df, y):
    start = time.time()
    vals_q = {}
    for i in model_pool_temp_path:
        
        model_pool_temp_path = copy.deepcopy(model_pool_temp_path)
        model_pool_temp_path.remove(i)
        model_pool_temp = [keras.models.load_model(i) for i in model_pool_temp_path]
        vals_q[i] = compute_divS(model_pool_temp, df, y)
    to_remove = min(vals_q, key=vals_q.get)
    #print('removing ', to_remove)
    model_pool.remove(to_remove)
    print('call to compute_difference lasted', time.time()-start)
    return model_pool

def Q(df, y, m1, m2):
    start = time.time()
    y_prob = m1.predict(df)
    p1 = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
    
    y_prob = m2.predict(df)
    p2 = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
    
    # matrice des resultats
    r = pd.DataFrame()
    r['y'] = y
    r['m1'] = p1
    r['m2'] = p2
    r['m1'] = (r['y']==r['m1']).astype(int)
    r['m2'] = (r['y']==r['m2']).astype(int)
    N00 = max(1, len(r[(r.m1==0) & (r.m2==0)]))
    N10 = max(1, len(r[(r.m1==1) & (r.m2==0)]))
    N01 = max(1, len(r[(r.m1==0) & (r.m2==1)]))
    N11 = max(1, len(r[(r.m1==1) & (r.m2==1)]))
    print('call to Q lasted', time.time()-start)
    return (N00*N11-N01*N10)/(N00*N11+N01*N10)

In [4]:
thetas_g = [10, 8, 6, 8, 10, 12, 14, 12]

def class_function(df, theta, noise=True):
    y = ((df[0]+df[1])<=theta).astype(int)
    if noise:
        for i in y.sample(frac=0.1).index:
            y.loc[i] = random.randint(0, 1)
    return y

def buid_set(size, generator, theta):
    X, Y, X_ = [], [], []
    for i in range(size):
        x, y = generator.next_sample()
        X_.append(x[0])
        X.append(x)
        #Y.append(float(y[0]))

    Y = class_function(pd.DataFrame(X_), theta, noise=True)
    Y = pd.get_dummies(pd.Series(Y)).to_numpy()
    return np.array(X), np.array(Y)

generator = sea()


X_train, y_train = buid_set(10000, generator, 10)
X_test, y_test = buid_set(1000, generator, 10)

In [8]:
class dtel:
    def __init__(self, pool_size=4):
        self.pool_size=pool_size
        self.pool = []
        self.new_pool = []
        self.model_pool_weight = []
        self.model_pool_paths = []
    
    def train_new_model(self, df, y):
        # update all models
        self.new_pool = copy.deepcopy(self.pool[:])
        
        
        self.temp_pool = []
        # for all models
        #print('updating models')
        
        for path in self.model_pool_paths:
            start = time.time()
            model = keras.models.load_model(path)
            #freeze all layers except last one
            for i in model.layers[:-1]:
                i.trainable = False
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            model.fit(df, y, epochs=20, batch_size=100, verbose=0)
            print('retrain lasted ', time.time()-start)
            self.temp_pool.append(path)
            
        # train new model
        #print('training models')
        start = time.time()
        new_model = create_model()
        new_model.fit(df, y, epochs=20, batch_size=100, verbose=0)
        print('train from scratch lasted ', time.time()-start)
        m_name = 'models/model-'+str(random.randint(0, 100000000))
        new_model.save(m_name)
        keras.backend.clear_session()
        self.model_pool_paths.append(m_name)
        self.temp_pool.append(m_name)
        # compute weights
        self.model_pool_weight = mse_i_t(self.temp_pool, df)
        
    def update_pool(self, df, y):
        if len(self.temp_pool) > self.pool_size:
            self.pool = compute_difference(self.pool, df, y)
        else :
            self.new_pool.extend([self.temp_pool[-1]])
            self.pool = list(self.new_pool[:])

    def predict(self, df):
        y_prob = keras.models.load_model(self.temp_pool[0]).predict(df)
        pred = np.array([np.argmax(y_prob[i]) for i in range(len(y_prob))])*self.model_pool_weight[0]
        
        for i in range(1, len(self.temp_pool)):
            y_prob = keras.models.load_model(self.temp_pool[0]).predict(df)
            pred = pred+np.array([np.argmax(y_prob[j]) for j in range(len(y_prob))])*self.model_pool_weight[i]

        pred = pred/sum(self.model_pool_weight)
        return pred

In [21]:
m = dt.pool[0]

In [22]:
m.predict(X_train)

array([[0.44347575, 0.55652434],
       [0.6083661 , 0.39163393],
       [0.4267288 , 0.5732712 ],
       [0.49706712, 0.50293285],
       [0.6282034 , 0.37179664],
       [0.33017904, 0.66982096],
       [0.5143756 , 0.48562434],
       [0.50209856, 0.49790147],
       [0.45032465, 0.54967535],
       [0.411067  , 0.58893305],
       [0.41390935, 0.5860906 ],
       [0.59240204, 0.40759796],
       [0.53431225, 0.46568772],
       [0.62512136, 0.37487862],
       [0.52128917, 0.4787108 ],
       [0.50159115, 0.49840888],
       [0.5757906 , 0.42420945],
       [0.57401043, 0.42598963],
       [0.5308843 , 0.46911573],
       [0.59802055, 0.40197948],
       [0.52030057, 0.4796995 ],
       [0.43330497, 0.56669503],
       [0.57886595, 0.4211341 ],
       [0.6246015 , 0.37539846],
       [0.52389055, 0.47610942],
       [0.55535036, 0.44464964],
       [0.6374854 , 0.36251467],
       [0.5692354 , 0.43076462],
       [0.48996332, 0.51003677],
       [0.5105023 , 0.48949775],
       [0.

In [25]:
dt.temp_pool

[<keras.engine.sequential.Sequential at 0x7f7240436748>, 'model-53633328']

In [26]:
dt.pool

['model-21181371', 'model-53633328']

In [6]:
tf.executing_eagerly()

False

In [9]:
%%time
dt = dtel(pool_size=25)

r, preds, results = [], [], []
correct_cnt = 0
pool = []
size = 500
start = t.time()
for i in range(120):
    
    theta = thetas_g[i//15]
    X_train, y_train = buid_set(size, generator, theta)
    if i != 0:
        y_pred = dt.predict(X_train)
        correct_cnt += sum(pd.DataFrame(y_pred)[0]==pd.DataFrame(y_train)[1])
        preds.extend(y_pred)
        results.append(sum(pd.DataFrame(y_pred)[0]==pd.DataFrame(y_train)[1])/size)
        print(i, results[-1], t.time()-start)
        start = t.time()
    dt.train_new_model(X_train, y_train)
    dt.update_pool(X_train, y_train)
    pool.append(dt.pool)

train from scratch lasted  1.1466691493988037
call to mse_i_t lasted 3.337860107421875e-06
1 0.482 1.997910499572754
retrain lasted  1.818281650543213
train from scratch lasted  1.1399996280670166
call to compute_weights lasted 0.07353925704956055
call to mse_i_t lasted 0.6964309215545654
2 0.47 5.470816612243652
retrain lasted  1.925886869430542
retrain lasted  2.126603126525879
train from scratch lasted  1.141930103302002
call to compute_weights lasted 0.07512998580932617
call to compute_weights lasted 0.08815789222717285
call to mse_i_t lasted 1.4974651336669922
3 0.506 9.787433862686157
retrain lasted  2.4724528789520264
retrain lasted  2.5817830562591553
retrain lasted  2.8277230262756348
train from scratch lasted  1.1543269157409668
call to compute_weights lasted 0.07506251335144043
call to compute_weights lasted 0.08895134925842285
call to compute_weights lasted 0.10768008232116699
call to mse_i_t lasted 2.4419095516204834
4 0.488 16.32676601409912


KeyboardInterrupt: 

In [42]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1158067292325824233
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17123479464010802845
physical_device_desc: "device: XLA_CPU device"
]


In [24]:
pool

[[<keras.engine.sequential.Sequential at 0x7f6ade1c26d8>],

In [26]:
pool[0][0].get_weights()

[array([[-0.09237836,  0.06928014, -0.1154742 , -0.05878377, -0.02426073],
        [ 0.04625523,  0.05701505,  0.00864731, -0.00980854, -0.0073113 ],
        [ 0.00019815, -0.0056593 ,  0.03725636, -0.03147896,  0.07602184]],
       dtype=float32),
 array([ 0.02094292, -0.02009555,  0.02109781,  0.        ,  0.02090373],
       dtype=float32),
 array([[-0.00924385,  0.12908757],
        [ 0.02675428, -0.07053556],
        [-0.05411606,  0.08099977],
        [ 0.00080629,  0.05344507],
        [-0.04452569,  0.01266712]], dtype=float32),
 array([-0.0196235,  0.0196235], dtype=float32)]

In [27]:
pool[1][0].get_weights()

[array([[-0.09237836,  0.06928014, -0.1154742 , -0.05878377, -0.02426073],
        [ 0.04625523,  0.05701505,  0.00864731, -0.00980854, -0.0073113 ],
        [ 0.00019815, -0.0056593 ,  0.03725636, -0.03147896,  0.07602184]],
       dtype=float32),
 array([ 0.02094292, -0.02009555,  0.02109781,  0.        ,  0.02090373],
       dtype=float32),
 array([[-0.00924385,  0.12908757],
        [ 0.02675428, -0.07053556],
        [-0.05411606,  0.08099977],
        [ 0.00080629,  0.05344507],
        [-0.04452569,  0.01266712]], dtype=float32),
 array([-0.0196235,  0.0196235], dtype=float32)]

In [29]:
pool[2][0].get_weights()

[array([[-0.09237836,  0.06928014, -0.1154742 , -0.05878377, -0.02426073],
        [ 0.04625523,  0.05701505,  0.00864731, -0.00980854, -0.0073113 ],
        [ 0.00019815, -0.0056593 ,  0.03725636, -0.03147896,  0.07602184]],
       dtype=float32),
 array([ 0.02094292, -0.02009555,  0.02109781,  0.        ,  0.02090373],
       dtype=float32),
 array([[-0.00924385,  0.12908757],
        [ 0.02675428, -0.07053556],
        [-0.05411606,  0.08099977],
        [ 0.00080629,  0.05344507],
        [-0.04452569,  0.01266712]], dtype=float32),
 array([-0.0196235,  0.0196235], dtype=float32)]

In [23]:
dt.pool[0].get_weights()

[array([[-0.09237836,  0.06928014, -0.1154742 , -0.05878377, -0.02426073],
        [ 0.04625523,  0.05701505,  0.00864731, -0.00980854, -0.0073113 ],
        [ 0.00019815, -0.0056593 ,  0.03725636, -0.03147896,  0.07602184]],
       dtype=float32),
 array([ 0.02094292, -0.02009555,  0.02109781,  0.        ,  0.02090373],
       dtype=float32),
 array([[-0.00924385,  0.12908757],
        [ 0.02675428, -0.07053556],
        [-0.05411606,  0.08099977],
        [ 0.00080629,  0.05344507],
        [-0.04452569,  0.01266712]], dtype=float32),
 array([-0.0196235,  0.0196235], dtype=float32)]

In [21]:
dt.pool[0].get_weights()

[array([[ 0.16694741,  0.12433422, -0.02733341,  0.01936467, -0.06072316],
        [ 0.0830221 ,  0.05574498, -0.0251658 ,  0.04658831,  0.02046689],
        [-0.06261133, -0.03913963,  0.07213517,  0.01632791,  0.0440337 ]],
       dtype=float32),
 array([-0.02040634, -0.02020923,  0.02034671,  0.0101704 ,  0.02005466],
       dtype=float32),
 array([[ 0.05844754,  0.00856236],
        [ 0.0389089 , -0.02927567],
        [-0.03226162,  0.03062434],
        [ 0.0567523 ,  0.04109545],
        [-0.15652652,  0.0207536 ]], dtype=float32),
 array([-0.01956593,  0.01956593], dtype=float32)]

In [ ]:
print(results[-1])

In [106]:
model.fit(X_train, y_train, )

Epoch 1/1
10000/10000 [==============================] - 1s 146us/step - loss: 0.5855 - acc: 0.6771


In [23]:
epochs = 3
batch_size = 128
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 10000 samples, validate on 1000 samples
Epoch 1/3





10000/10000 [==============================] - 1s 148us/step - loss: 0.6794 - acc: 0.6730 - val_loss: 0.6654 - val_acc: 0.6770
Epoch 2/3
10000/10000 [==============================] - 0s 18us/step - loss: 0.6482 - acc: 0.7328 - val_loss: 0.6330 - val_acc: 0.7660
Epoch 3/3
10000/10000 [==============================] - 0s 20us/step - loss: 0.6125 - acc: 0.7818 - val_loss: 0.5969 - val_acc: 0.7770


In [45]:
model.trainable_weights

[<tf.Variable 'dense_4/kernel:0' shape=(3, 3) dtype=float32_ref>,
 <tf.Variable 'dense_4/bias:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(3, 2) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(2,) dtype=float32_ref>]

In [52]:
model.weights[0].trainable = False

AttributeError: can't set attribute

In [51]:
l

AttributeError: can't set attribute

In [8]:
d.trainable = False

In [9]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 785       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 4         
Total params: 789
Trainable params: 4
Non-trainable params: 785
_________________________________________________________________


In [12]:
epochs = 2
batch_size = 128
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 2s 36us/step - loss: 0.1084 - acc: 0.9680 - val_loss: 0.1012 - val_acc: 0.9711
Epoch 2/2
60000/60000 [==============================] - 1s 23us/step - loss: 0.1035 - acc: 0.9683 - val_loss: 0.0986 - val_acc: 0.9714


In [13]:
after_3= d.get_weights()

In [32]:
d.trainable = True
print(model1.trainable_weights)

epochs = 6
batch_size = 128
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

[<tf.Variable 'dense_1/kernel:0' shape=(784, 1) dtype=float32_ref>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>, <tf.Variable 'dense_2/kernel:0' shape=(1, 2) dtype=float32_ref>, <tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32_ref>]
Train on 60000 samples, validate on 10000 samples
Epoch 1/6
60000/60000 [==============================] - 1s 22us/step - loss: 0.0981 - acc: 0.9718 - val_loss: 0.0926 - val_acc: 0.9733
Epoch 2/6
60000/60000 [==============================] - 1s 22us/step - loss: 0.0944 - acc: 0.9728 - val_loss: 0.0917 - val_acc: 0.9726
Epoch 3/6
60000/60000 [==============================] - 1s 22us/step - loss: 0.0917 - acc: 0.9736 - val_loss: 0.0884 - val_acc: 0.9747
Epoch 4/6
60000/60000 [==============================] - 1s 22us/step - loss: 0.0899 - acc: 0.9739 - val_loss: 0.0888 - val_acc: 0.9752
Epoch 5/6
60000/60000 [==============================] - 1s 22us/step - loss: 0.0888 - acc: 0.9743 - val_loss: 0.0881 - val_acc: 0.9756
Epoch 6/6
60000

In [16]:
after[0] == after_3[0]

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [18]:
after[0][0:20]

array([[ 0.04466002],
       [ 0.04611144],
       [ 0.03130344],
       [-0.03725884],
       [ 0.01148854],
       [ 0.00646817],
       [-0.01815568],
       [ 0.00515334],
       [-0.02820878],
       [ 0.01650673],
       [-0.01893103],
       [-0.0122538 ],
       [-0.05547793],
       [ 0.07850227],
       [-0.03446824],
       [ 0.08233008],
       [-0.07949447],
       [-0.02539072],
       [ 0.05617425],
       [ 0.04319464]], dtype=float32)

In [15]:
l = model1.trainable_weights[0]

In [27]:
model1.trainable_weights

[<tf.Variable 'dense_2/kernel:0' shape=(1, 2) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32_ref>]

False

In [19]:
l.graph

In [20]:
l.initial_value

<tf.Tensor 'dense_5/random_normal:0' shape=(784, 1) dtype=float32>

In [22]:
?l.read_value

Signature: l.read_value()
Docstring:
Returns the value of this variable, read in the current context.

Can be different from value() if it's on another device, with control
dependencies, etc.

Returns:
  A `Tensor` containing the value of the variable.
File:      ~/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py
Type:      method


In [ ]:
l.